In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
df = pd.read_csv(r"")
X = df.drop("d33",axis=1)
y = df['d33']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)

In [ ]:
# Shuffle columns using numpy's random.permutation
np.random.seed(0)  # Random seed to ensure reproducible results
new_columns = df.columns[np.random.permutation(len(df.columns))]
dff = df[new_columns]

In [ ]:
import catboost as cb
from sklearn.model_selection import GridSearchCV

# CatBoost model parameters
params_catboost = {
    'learning_rate': 0.02,            # Learning rate, which controls the step size of each step
    'loss_function': 'RMSE',          # Loss function, where RMSE is used
    'eval_metric': 'RMSE',            # Evaluation metric, where RMSE is used
    'random_seed': 42,                # Random seed
    'verbose': 100,                   # The detail level of CatBoost's output information and output the progress every 100 iterations
    'thread_count': -1,               # Use all available CPU cores
    'bootstrap_type': 'Bernoulli',    # Bootstrap type, which is used to control the generalization ability of the model
    'subsample': 0.7,                 # The proportion of samples randomly selected in each iteration
    'colsample_bylevel': 0.6          # The proportion of features randomly selected at each layer, which is used to increase the generalization ability of the model
}

# Initialize the CatBoost model
model_catboost = cb.CatBoostRegressor(**params_catboost)

# Define a parameter grid for grid search
param_grid = {
    'iterations': [100, 200, 300, 400, 500],  # Number of iterations (equivalent to the number of trees)
    'depth': [3, 4, 5, 6, 7],                 # Tree depth
    'learning_rate': [0.01, 0.02, 0.05, 0.1], # Learning rate
}

# Use GridSearchCV for grid search and k-fold cross-validation
grid_search = GridSearchCV(
    estimator=model_catboost,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Evaluation metric is negative MSE
    cv=10,                             # 10-fold cross-validation
    n_jobs=-1,                         # Parallel computing
)

# Model training
grid_search.fit(X_train, y_train)

# Output the optimal parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best RMSE score: ", (-grid_search.best_score_) ** 0.5)  # Restore RMSE

# Train the model with optimal parameters
best_model_catboost = grid_search.best_estimator_

0:	learn: 119.3396221	total: 133ms	remaining: 1m 6s
100:	learn: 58.0379618	total: 2.05s	remaining: 8.09s
200:	learn: 44.9506754	total: 3.94s	remaining: 5.86s
300:	learn: 36.7959167	total: 5.66s	remaining: 3.74s
400:	learn: 31.3698563	total: 7.39s	remaining: 1.82s
499:	learn: 27.0313796	total: 9.15s	remaining: 0us
Best parameters found:  {'depth': 7, 'iterations': 500, 'learning_rate': 0.05}
Best RMSE score:  74.0756700038408


In [ ]:
# Obtain and sort the feature importance of CatBoost
catboost_feature_importances = best_model_catboost.get_feature_importance()
catboost_sorted_indices = np.argsort(catboost_feature_importances)[::-1]
catboost_sorted_features = X_train.columns[catboost_sorted_indices]
catboost_sorted_importances = catboost_feature_importances[catboost_sorted_indices]

In [ ]:
# Get feature importance
feature_importances = best_model_catboost.feature_importances_

# Construct a feature importance ranking
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Take the top 20 important features
top_n = 20
top_features = feature_importance_df.head(top_n)

# Adjust the font size
plt.figure(figsize=(12, 8), dpi=1000)  
plt.barh(top_features['Feature'], top_features['Importance'], color='skyblue')
plt.xlabel('Importance', fontsize=14)  
plt.ylabel('Feature', fontsize=14)  
plt.title(f'Top {top_n} Feature Importance', fontsize=16)  
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.gca().invert_yaxis()  
plt.savefig("1.png", format='png', bbox_inches='tight')
plt.show()

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

# LightGBM model parameters
params_lgb = {
    'boosting_type': 'gbdt',           # Boosting method, here using Gradient Boosting Decision Trees (GBDT)
    'objective': 'regression',         # Loss function, here using it for regression tasks
    'num_leaves': 127,                 # Number of leaf nodes per tree, which controls model complexity
    'verbosity': 1,                    # Control the detail level of LightGBM's output information
    'seed': 42,                        # Random seed, used to reproduce model results
    'n_jobs': -1,                      # Number of threads for parallel computing, where -1 indicates using all available CPU cores
    'colsample_bytree': 0.6,           # Proportion of features randomly selected for each tree, used to enhance the model's generalization ability
    'subsample': 0.7,                  # Proportion of samples randomly selected in each iteration, used to enhance the model's generalization ability
    'metric': 'rmse'                   # Evaluation metric, here using Root Mean Square Error (RMSE)
}

# Initialize the LightGBM model
model_lgb = lgb.LGBMRegressor(**params_lgb)

# Define a parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  # Number of trees
    'max_depth': [3, 4, 5, 6, 7],               # Tree depth
    'learning_rate': [0.01, 0.02, 0.05, 0.1],   # Learning rate
}

# Use GridSearchCV for grid search and k-fold cross-validation
grid_search = GridSearchCV(
    estimator=model_lgb,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Evaluation metric is negative MSE
    cv=10,                             # 10-fold cross-validation
    n_jobs=-1,                         # Parallel computing
    verbose=1                          # Output detailed progress information
)

# Model training
grid_search.fit(X_train, y_train)

# Output the optimal parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best RMSE score: ", (-grid_search.best_score_) ** 0.5)  # Restore RMSE


# Train the model with optimal parameters
best_model_lgb = grid_search.best_estimator_

In [ ]:
# Obtain and sort the feature importance of LightGBM
lgb_feature_importances = best_model_lgb.feature_importances_
lgb_sorted_indices = np.argsort(lgb_feature_importances)[::-1]
lgb_sorted_features = X_train.columns[lgb_sorted_indices]
lgb_sorted_importances = lgb_feature_importances[lgb_sorted_indices]

In [ ]:
# Get feature importance
feature_importances = best_model_lgb.feature_importances_

# Construct a feature importance ranking
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Take the top 20 important features
top_n = 20
top_features = feature_importance_df.head(top_n)

# Adjust the font size
plt.figure(figsize=(12, 8), dpi=1000)  
plt.barh(top_features['Feature'], top_features['Importance'], color='skyblue')
plt.xlabel('Importance', fontsize=14)  
plt.ylabel('Feature', fontsize=14)  
plt.title(f'Top {top_n} Feature Importance', fontsize=16)  
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.gca().invert_yaxis()  
plt.savefig("1.png", format='png', bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# RandomForestRegressor model parameters
params_rfr = {
    'criterion' : 'squared_error', # The quality metric used for splitting. 'squared_error' indicates the use of MSE.
    'min_weight_fraction_leaf' : 0.0, # Similar to 'min_samples_leaf', but based on the total sample weights.
    'random_state' : 42,          # The random number generation to ensure reproducible results.
    'max_leaf_nodes' : None,      # The maximum number of leaf nodes per tree. 'None' indicates no limit.
}

# Initialize the RandomForestRegressor model
model_rfr = RandomForestRegressor(**params_rfr)

# Define a parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500], # Number of trees
    'max_depth': [None, 3, 5, 7], # Maximum depth of each tree. 'None' indicates no depth limit.
    'min_samples_split': [2, 3, 4, 5, 6], # Minimum number of samples required for a node to split.
    'min_samples_leaf': [1, 2, 3, 4, 5] # Minimum number of samples required for a leaf node.

}

# Use GridSearchCV for grid search and k-fold cross-validation
grid_search = GridSearchCV(
    estimator=model_rfr,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Evaluation metric is negative MSE
    cv=10,                             # 10-fold cross-validation
    n_jobs=-1,                         # Parallel computing
    verbose=1                          # Output detailed progress information
)

# Model training
grid_search.fit(X_train, y_train)

# Output the optimal parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best RMSE score: ", (-grid_search.best_score_) ** 0.5)  # Restore RMSE

# Train the model with optimal parameters
best_model_rfr = grid_search.best_estimator_

In [ ]:
# Obtain and sort the feature importance of RandomForest
rf_feature_importances = best_model_rfr.feature_importances_
rf_sorted_indices = np.argsort(rf_feature_importances)[::-1]
rf_sorted_features = X_train.columns[rf_sorted_indices]
rf_sorted_importances = rf_feature_importances[rf_sorted_indices]

In [ ]:
# Get feature importance
feature_importances = best_model_rfr.feature_importances_

# Construct a feature importance ranking
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Take the top 20 important features
top_n = 20
top_features = feature_importance_df.head(top_n)

# Adjust the font size
plt.figure(figsize=(12, 8), dpi=1000)  
plt.barh(top_features['Feature'], top_features['Importance'], color='skyblue')
plt.xlabel('Importance', fontsize=14)  
plt.ylabel('Feature', fontsize=14)  
plt.title(f'Top {top_n} Feature Importance', fontsize=16)  
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.gca().invert_yaxis()  
plt.savefig("1.png", format='png', bbox_inches='tight')
plt.show()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# XGBoost model parameters
params_xgb = {
    'booster': 'gbtree',              # Boosting method, here using Gradient Boosting Tree.
    'objective': 'reg:squarederror',  # Loss function, here using squared error, which is suitable for regression tasks.
    'max_leaves': 127,                # The number of leaf nodes per tree, which controls the model complexity.
    'verbosity': 1,                   # The verbosity of XGBoost's output. 0 means no output, and 1 means outputting progress information.
    'seed': 42,                       # Random seed, used to reproduce the model's results.
    'nthread': -1,                    # The number of threads for parallel computation, where -1 indicates using all available CPU cores.
    'colsample_bytree': 0.6,          # The proportion of features randomly selected for each tree, used to enhance the model's generalization ability.
    'subsample': 0.7,                 # The proportion of samples randomly selected in each iteration, used to enhance the model's generalization ability.
    'eval_metric': 'rmse'             # Evaluation metric, here using RMSE.
}

# nitialize the XGBoost model
model_xgb = xgb.XGBRegressor(**params_xgb)

# Define a parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  # Number of trees.
    'max_depth': [3, 4, 5, 6, 7],               # Depth of the tree.
    'learning_rate': [0.01, 0.02, 0.05, 0.1],   # Learning rate.
}

# Use GridSearchCV for grid search and k-fold cross-validation
grid_search = GridSearchCV(
    estimator=model_xgb,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Evaluation metric is negative mean squared error
    cv=10,                             # 10-fold cross-validation
    n_jobs=-1,                         # Parallel computing
    verbose=1                          # Output detailed progress information
)

# Model training
grid_search.fit(X_train, y_train)

# Output the optimal parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best RMSE score: ", (-grid_search.best_score_) ** 0.5)  # 还原RMSE

# Train the model with optimal parameters
best_model_xgboost = grid_search.best_estimator_

In [ ]:
# Obtain and sort the feature importance of XGBoost
xgb_feature_importances = best_model_xgboost.feature_importances_
xgb_sorted_indices = np.argsort(xgb_feature_importances)[::-1]
xgb_sorted_features = X_train.columns[xgb_sorted_indices]
xgb_sorted_importances = xgb_feature_importances[xgb_sorted_indices]

In [ ]:
# Get feature importance
feature_importances = best_model_xgboost.feature_importances_

# Construct a feature importance ranking
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Take the top 20 important features
top_n = 20
top_features = feature_importance_df.head(top_n)

# Adjust the font size
plt.figure(figsize=(12, 8), dpi=1000)  
plt.barh(top_features['Feature'], top_features['Importance'], color='skyblue')
plt.xlabel('Importance', fontsize=14)  
plt.ylabel('Feature', fontsize=14)  
plt.title(f'Top {top_n} Feature Importance', fontsize=16)  
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.gca().invert_yaxis()  
plt.savefig("1.png", format='png', bbox_inches='tight')
plt.show()

In [ ]:
# DataFrame to store the feature importance of all models.
feature_importance_df = pd.DataFrame({
    "RandomForest_Feature": rf_sorted_features,
    "RandomForest_Importance": rf_sorted_importances,
    "XGBoost_Feature": xgb_sorted_features,
    "XGBoost_Importance": xgb_sorted_importances,
    "LightGBM_Feature": lgb_sorted_features,
    "LightGBM_Importance": lgb_sorted_importances,
    "CatBoost_Feature": catboost_sorted_features,
    "CatBoost_Importance": catboost_sorted_importances
})
feature_importance_df